<a href="https://colab.research.google.com/github/algikckyvz/Food-Classification/blob/main/ASSIGNMENT1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/CMP719-CV/archive.zip','r') as z:
  z.extractall(".")

In [3]:
import torch
from torch.utils.data import Dataset

from PIL import Image

class LoaderClass(Dataset):
    def __init__(self,data,labels,phase,transforms):
        super(LoaderClass, self).__init__()
        self.transforms = transforms
        self.labels = labels[phase + "_labels"]
        self.data = data[phase + "_data"]
        self.phase = phase



    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]
        img = self.data[idx]
        img = Image.fromarray(img)
        img = self.transforms(img)
        return img,torch.from_numpy(label)

In [2]:
%cd '/content/drive/MyDrive/Colab Notebooks/CV/ASSIGNMENT1'
from tensorflow.train import Feature
from torch.utils.data import DataLoader
import torch
from torch.nn import CrossEntropyLoss , L1Loss , NLLLoss
from torch import optim
from torchvision import transforms, utils
import numpy as np
import os
from PIL import Image,ImageFile
import matplotlib.pyplot as plt

/content/drive/MyDrive/Colab Notebooks/CV/ASSIGNMENT1


In [4]:
DATA_PATH = '/content/'
NUM_FOOD = 11

evaluation_data = []
evaluation_labels = []

validation_data = []
validation_labels = []

training_data = []
training_labels = []

# PARSING EVALUTION IMAGES    
# PARSE THE NAME LESS THAN 1 AND APPEND TO LIST  
path_evaluation = os.path.join(DATA_PATH, "evaluation")
FOOD = [x for x in os.listdir(path_evaluation) if x != '.config']#'.DS_Store'
print(FOOD)
for folder in os.listdir(path_evaluation):
  print(folder)
  class_index = FOOD.index(folder)
  print(class_index)
  for image_name in os.listdir(path_evaluation + "/" + folder):
    if int(os.path.splitext(image_name)[0]) < 1:
      img = Image.open(path_evaluation + "/" + folder + "/" + image_name)
      food_label = (np.arange(NUM_FOOD) == class_index).astype(np.float32)
      evaluation_data.append(np.array(img))
      evaluation_labels.append(food_label) 

['Meat', 'Seafood', 'Vegetable-Fruit', 'Soup', 'Dairy product', 'Egg', 'Noodles-Pasta', 'Rice', 'Dessert', 'Bread', 'Fried food']
Meat
0
Seafood
1
Vegetable-Fruit
2
Soup
3
Dairy product
4
Egg
5
Noodles-Pasta
6
Rice
7
Dessert
8
Bread
9
Fried food
10


In [5]:
# IMPORT IMAGES FROM FOLDERS RANDOMLY. THIS IMPROVES VALIDATION ACCURACY. 
import random 
from random import seed,sample

DATA_PATH = '/content/'

path_validation = os.path.join(DATA_PATH, "validation")
for folder in os.listdir(path_validation):
  class_index = FOOD.index(folder)
  seed(1)
  image_count = [i for i in os.listdir(path_validation + "/" + folder)]
  print(image_count)

  if len(image_count) < 1200:
    subset = sample(image_count, len(image_count))
  else: 
    subset =sample(image_count, 1200)
  print(subset)

  for image_name in subset:
    img = Image.open(path_validation + "/" + folder + "/" + image_name)
    food_label = (np.arange(NUM_FOOD) == class_index).astype(np.float32)
    validation_data.append(np.array(img))
    validation_labels.append(food_label)

['72.jpg', '75.jpg', '441.jpg', '30.jpg', '310.jpg', '183.jpg', '33.jpg', '109.jpg', '107.jpg', '62.jpg', '163.jpg', '283.jpg', '87.jpg', '303.jpg', '92.jpg', '147.jpg', '227.jpg', '100.jpg', '344.jpg', '307.jpg', '364.jpg', '162.jpg', '34.jpg', '407.jpg', '102.jpg', '383.jpg', '249.jpg', '250.jpg', '354.jpg', '398.jpg', '206.jpg', '448.jpg', '281.jpg', '26.jpg', '374.jpg', '18.jpg', '425.jpg', '5.jpg', '381.jpg', '265.jpg', '171.jpg', '35.jpg', '24.jpg', '274.jpg', '431.jpg', '143.jpg', '299.jpg', '446.jpg', '358.jpg', '415.jpg', '184.jpg', '309.jpg', '55.jpg', '210.jpg', '83.jpg', '151.jpg', '192.jpg', '111.jpg', '376.jpg', '73.jpg', '115.jpg', '298.jpg', '108.jpg', '345.jpg', '37.jpg', '159.jpg', '17.jpg', '233.jpg', '325.jpg', '8.jpg', '116.jpg', '308.jpg', '89.jpg', '127.jpg', '335.jpg', '275.jpg', '315.jpg', '355.jpg', '22.jpg', '150.jpg', '405.jpg', '114.jpg', '47.jpg', '392.jpg', '444.jpg', '320.jpg', '244.jpg', '430.jpg', '211.jpg', '359.jpg', '409.jpg', '132.jpg', '336.jpg', 

In [6]:
# PARSE IMAGES THAT WE USE FOR TRAINING. 

DATA_PATH = '/content/'

path_training = os.path.join(DATA_PATH, "training")
for folder in os.listdir(path_training):
  class_index = FOOD.index(folder)
  seed(1) # I wanted to fixted randomization for clarifity of the accuracy but it can be closed. 
  image_count = [i for i in os.listdir(path_training + "/" + folder)]
  print(image_count)
  if len(image_count) < 350:
    subset = sample(image_count, len(image_count))
  else: 
    subset = sample(image_count, 350)
  print(subset)
  for image_name in subset:
    img = Image.open(path_training + "/" + folder + "/" + image_name)
    food_label = (np.arange(NUM_FOOD) == class_index).astype(np.float32)
    training_data.append(np.array(img))
    training_labels.append(food_label)

['535.jpg', '72.jpg', '75.jpg', '441.jpg', '851.jpg', '549.jpg', '30.jpg', '1158.jpg', '1038.jpg', '809.jpg', '567.jpg', '720.jpg', '534.jpg', '310.jpg', '493.jpg', '1213.jpg', '1013.jpg', '183.jpg', '522.jpg', '1079.jpg', '33.jpg', '1289.jpg', '569.jpg', '109.jpg', '934.jpg', '945.jpg', '554.jpg', '107.jpg', '1267.jpg', '62.jpg', '163.jpg', '1104.jpg', '1059.jpg', '1165.jpg', '898.jpg', '283.jpg', '87.jpg', '1061.jpg', '303.jpg', '92.jpg', '1264.jpg', '982.jpg', '147.jpg', '227.jpg', '100.jpg', '344.jpg', '818.jpg', '1207.jpg', '1171.jpg', '1309.jpg', '728.jpg', '667.jpg', '307.jpg', '1282.jpg', '364.jpg', '1133.jpg', '985.jpg', '518.jpg', '162.jpg', '577.jpg', '34.jpg', '407.jpg', '537.jpg', '761.jpg', '102.jpg', '1076.jpg', '383.jpg', '1241.jpg', '249.jpg', '870.jpg', '460.jpg', '1018.jpg', '250.jpg', '354.jpg', '453.jpg', '398.jpg', '206.jpg', '1021.jpg', '923.jpg', '448.jpg', '281.jpg', '1123.jpg', '1191.jpg', '26.jpg', '374.jpg', '783.jpg', '18.jpg', '1089.jpg', '723.jpg', '1239.

In [7]:
data = {"train_data":training_data,"val_data":validation_data}
labels = {"train_labels":np.array(training_labels),"val_labels":np.array(validation_labels)}
evaluation_dict = {"evaluation_data":evaluation_data,"evaluation_labels":evaluation_labels}

In [8]:
len_train = len(data["train_data"])
print("Number of images in training data : {}".format(len_train))

len_validation = len(data["val_data"])
print("Number of images in validation data : {}".format(len_validation))

len_evaluation = len(evaluation_dict["evaluation_data"])
print("Number of images in evaluation data : {}".format(len_evaluation))

Number of images in training data : 3780
Number of images in validation data : 3430
Number of images in evaluation data : 11


In [9]:
# VISDOM is used to visulize run time training and validation accuracies. 
!pip install visdom

In [10]:
import torch
from torch.nn import CrossEntropyLoss
from torchvision import datasets, models
from torch import optim
from torchvision import transforms, utils
import numpy as np
import os
from PIL import Image,ImageFile

LR = 1e-2
Momentum = 0.9 # If you use SGD with momentum
BATCH_SIZE = 8
USE_CUDA = False
POOLING = False
NUM_EPOCHS = 50
PATIENCE = -1
TRAIN_PERCENT = 0.7
VAL_PERCENT = 0.3
NUM_ARTISTS = 11
ImageFile.LOAD_TRUNCATED_IMAGES = True 

import torchvision.models as models
import numpy as np

In [11]:
import torch
from torch.utils.data import Dataset

from PIL import Image

class LoaderClass(Dataset):
    def __init__(self,data,labels,phase,transforms):
        super(LoaderClass, self).__init__()
        self.transforms = transforms
        self.labels = labels[phase + "_labels"]
        self.data = data[phase + "_data"]
        self.phase = phase

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]
        img = self.data[idx]
        img = Image.fromarray(img)
        img = self.transforms(img)
        return img,torch.from_numpy(label)

class SimpleCNN(torch.nn.Module):
    def __init__(self,use_cuda=False,pooling= False):
        super(SimpleCNN, self).__init__()
        self.use_cuda = use_cuda
        self.pooling = pooling
        self.conv_layer1 = torch.nn.Conv2d(in_channels=3,out_channels=12,kernel_size=5,stride=1)
        self.dropout_layer_1 = torch.nn.Dropout2d(0.5) # yeni eklendi
        self.pool_layer1 = torch.nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv_layer2 = torch.nn.Conv2d(in_channels=12,out_channels=15,kernel_size=5,stride=1)
        self.pool_layer2 = torch.nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv_layer3 = torch.nn.Conv2d(in_channels=15,out_channels=12,kernel_size=5,stride=1)
        self.pool_layer3 = torch.nn.MaxPool2d(kernel_size=2,stride=2) 
        self.bathcnorm1 = torch.nn.BatchNorm2d(12)
        self.bathcnorm2 = torch.nn.BatchNorm2d(15)
        self.bathcnorm3 = torch.nn.BatchNorm2d(8) 
        if pooling:
          self.fully_connected_layer1 = nn.Linear(1215,256) 
          self.fully_connected_layer2 = nn.Linear(256,256)    
          self.bathcnormFC1 = torch.nn.BatchNorm1d(256)
          self.bathcnormFC2 = torch.nn.BatchNorm1d(256)         
          self.final_layer = nn.Linear(256,11) 
        else:
          self.bathcnormFC1 = torch.nn.BatchNorm1d(512)
          self.bathcnormFC2 = torch.nn.BatchNorm1d(512)
          self.fully_connected_layer1 = nn.Linear(26460,512) 
          self.fully_connected_layer2 = nn.Linear(512,512)           
          self.final_layer = nn.Linear(512, 11)
    def forward(self,inp):
        x = torch.nn.functional.relu(self.bathcnorm1(self.conv_layer1(inp)))
        #print('CONV1 OUT',x.shape)
        if self.pooling:
            x = self.pool_layer1(x)
            #print('CONV1+POOLING OUT',x.shape)
        x = torch.nn.functional.relu(self.bathcnorm2(self.conv_layer2(x)))
        #print('CONV2 OUT',x.shape)
        if self.pooling:
            x = self.pool_layer2(x)
            #print('CONV2+POOLING OUT',x.shape)
        #x = torch.nn.functional.relu(self.bathcnorm3(self.conv_layer3(x)))
        #if self.pooling: 
            #x = self.pool_layer3(x)
        #x = self.dropout_layer_1(x)
        x = x.reshape(x.size(0),-1)
        #print('DROPOUT OUT',x.shape)
        x = torch.nn.functional.relu(self.bathcnormFC1(self.fully_connected_layer1(x)))
        #print('FC1 OUT',x.shape)
        x = torch.nn.functional.relu(self.bathcnormFC2(self.fully_connected_layer2(x)))
        #print('FC2 OUT',x.shape)
        x = self.final_layer(x)
        return x

In [12]:
from torch import nn
import time
import torch
from torch.nn import CrossEntropyLoss
import tqdm
class Trainer():
    def __init__(self,model,criterion,tr_loader,val_loader,optimizer,num_epoch,patience,batch_size,lr_scheduler=None,display=None):
        self.model = model
        self.tr_loader = tr_loader
        self.val_loader = val_loader
        self.optimizer = optimizer
        self.num_epoch = num_epoch
        self.patience = patience
        self.lr_scheduler = lr_scheduler
        self.criterion = criterion
        self.softmax = nn.Softmax()
        self.no_inc = 0
        self.best_loss = 9999
        self.phases = ["train","val"]
        self.best_model = []
        self.best_val_acc = 0
        self.best_train_acc = 0
        self.best_val_loss = 0
        self.best_train_loss = 0
        self.batch_size = batch_size
        self.display = display
        self.model = model.to(torch.device(0))
        #if self.model.use_cuda:
            #self.model = self.model.cuda()


        pass
    def train(self):
        val_accuracies = [] # yeni
        val_win = None # yeni
        train_accuracies = [] # yeni
        train_win = None # yeni     
        pbar = tqdm.tqdm(desc= "Epoch 0, phase: Train",postfix="train_loss : ?, train_acc: ?")
        for i in range(self.num_epoch):
            last_train_acc = 0
            last_val_acc = 0
            last_val_loss = 0
            last_train_loss = 0
            pbar.update(1)

            for phase in self.phases:
                total_acc = 0
                total_loss = 0
                start = time.time()
                if phase == "train":
                    pbar.set_description_str("Epoch %d,"% i + "phase: Training")
                    loader = self.tr_loader
                    self.model.train()
                else:
                    pbar.set_description_str("Epoch %d,"% i + "phase: Validation")
                    loader = self.val_loader
                    self.model.eval()
                iter = 0
                for images,labels in loader:
                    iter += 1
                    #if self.model.use_cuda:
                    #images = images.to(0)
                    #labels = labels.to(0)
                    images = images.to(torch.device(0))
                    labels = labels.to(torch.device(0))

                    self.optimizer.zero_grad()
                    logits = self.model(images)
                    softmaxed_scores = self.softmax(logits)
                    _, predictions = torch.max(softmaxed_scores,1)
                    _, labels = torch.max(labels,1)
                    loss = self.criterion(softmaxed_scores.float(),labels.long())
                    total_loss += loss.item()
                    total_acc += torch.sum(predictions == labels).item()

                    if phase == "train":
                        pbar.set_postfix_str("train acc: %6.3f," %(total_acc/ (iter*self.batch_size)) + ("train loss: %6.3f" % (total_loss / iter)))
                        loss.backward()
                        self.optimizer.step()
                    else:
                        pbar.set_postfix_str("val acc: %6.3f," %(total_acc/ (iter*self.batch_size)) + ("val loss: %6.3f" % (total_loss / iter)))


                if phase == "train":
                    if self.lr_scheduler:

                        self.lr_scheduler.step(loss)
                end = time.time()
                if phase == "train":
                    loss_p = total_loss / iter
                    acc_p = total_acc / len(self.tr_loader.dataset)
                    last_train_acc = acc_p
                    last_train_loss = loss_p
                    # yeni eklendi
                    train_accuracies.append(last_train_acc)
                    if len(train_accuracies) > 0:
                      train_win = self.display.line(Y=np.array(train_accuracies),
                      X=np.arange(len(train_accuracies)),
                      win=train_win,
                      opts={
                            "title": "train accuracy",
                            "xlabel": "Epochs",
                            "ylabel": "Accuracy",
                        },)
                      # buraya kadar
                else:
                    loss_p = total_loss / iter
                    acc_p = total_acc / len(self.val_loader.dataset)
                    last_val_acc = acc_p
                    last_val_loss = loss_p
                    # yeni baslangic
                    val_accuracies.append(last_val_acc)
                    if len(val_accuracies) > 0:
                      val_win = self.display.line(Y=np.array(val_accuracies),
                      X=np.arange(len(val_accuracies)),
                      win=val_win,
                      opts={
                            "title": "Validation accuracy",
                            "xlabel": "Epochs",
                            "ylabel": "Accuracy",
                        },)
                    # yeni buraya kadar
                    if loss_p < self.best_loss:
                        print("New best loss, loss is: ",str(loss_p), "acc is: ",acc_p )
                        self.best_loss = loss_p
                        self.no_inc = 0
                        self.best_model = self.model
                        self.best_train_acc = last_train_acc
                        self.best_train_loss = last_train_loss
                        self.best_val_loss = last_val_loss
                        self.best_val_acc = last_val_acc
                    else:
                        print("Not a better score")
                        self.no_inc += 1
                        if self.no_inc == self.patience:
                            print("Out of patience returning the best model")
                            print(
                                "Best val acc: {}, Best val loss: {}, Best train acc: {}, Best train loss: {} ".format(
                                    self.best_val_acc, self.best_val_loss, self.best_train_acc, self.best_train_loss
                                ))  # Stats of the best model
                            return self.best_model
        print("Training ended returning the best model")
        print(
            "Best val acc: {}, Best val loss: {}, Best train acc: {}, Best train loss: {} ".format(
                self.best_val_acc, self.best_val_loss, self.best_train_acc, self.best_train_loss
            ))  # Stats of the best model
        return self.best_model
        
def train2(model, trainloader, criterion, optimizer, epochs = 5):
    train_loss =[]
    for e in range(epochs):
        running_loss =0
        for images, labels in trainloader:
            inputs, labels = images, labels

            optimizer.zero_grad()
            img = model(inputs)
            loss = criterion(img, torch.max(labels, 1)[1])
            running_loss+=loss
            loss.backward()
            optimizer.step()
        print("Epoch : {}/{}..".format(e+1,epochs),
         "Training Loss: {:.6f}".format(running_loss/len(trainloader))) 
        train_loss.append(running_loss)
    plt.plot(train_loss,label="Training Loss")
    plt.show() 

import torch
from torch.nn import CrossEntropyLoss
from torchvision import datasets, models
from torch import optim
from torchvision import transforms, utils
import numpy as np
import os
from PIL import Image,ImageFile

LR = 1e-2
Momentum = 0.9 # If you use SGD with momentum
BATCH_SIZE = 16
USE_CUDA = False
POOLING = False
NUM_EPOCHS = 50
PATIENCE = -1
TRAIN_PERCENT = 0.7
VAL_PERCENT = 0.3
NUM_ARTISTS = 11
ImageFile.LOAD_TRUNCATED_IMAGES = True # Do not change this

import torchvision.models as models
import numpy as np
if __name__ == "__main__":
    ! npm install -g localtunnel
    get_ipython().system_raw('python3 -m pip install visdom')
    get_ipython().system_raw('python3 -m visdom.server -port 6006 >> visdomlog.txt 2>&1 &')
    get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1 &')
    import time
    time.sleep(5)
    ! cat url.txt
    import visdom
    time.sleep(5)
    viz = visdom.Visdom(port='6006')
    
    ##### IF YOU WANT TO TRAIN FROM STRATCH COMMENT THIS LINES 
    model_transfer = models.resnet18(pretrained=True)
    num_ftrs = model_transfer.fc.in_features 
    model_transfer.fc = nn.Linear(num_ftrs, 11)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer_ft = optim.SGD(model_transfer.parameters(), lr=0.001, momentum=0.9)
    lr_sche = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
    ####
    transforms = {
        'train': transforms.Compose([
            transforms.Resize((50,50)),
            transforms.RandomResizedCrop(50),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(), 
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])           
            
        ]),
        'val': transforms.Compose([
            transforms.Resize((50,50)),
            transforms.CenterCrop(50),
            transforms.ToTensor(), 
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            
            
        ]),
        }
    train_dataset = LoaderClass(data,labels,"train",transforms["train"])
    valid_dataset = LoaderClass(data,labels,"val",transforms["val"])

    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=BATCH_SIZE,
                                               shuffle=True, num_workers=4, pin_memory=True)
    val_loader = torch.utils.data.DataLoader(valid_dataset,
                                             batch_size=BATCH_SIZE,
                                             shuffle=False, num_workers=4, pin_memory=True)
    ### IF YOU WANT TO TRAIN FROM STRATCH COMMENT THIS LINES 
    trainer_m = Trainer(model_transfer, criterion, train_loader, val_loader, optimizer_ft, num_epoch=200, patience=PATIENCE,batch_size=BATCH_SIZE,lr_scheduler= lr_sche,display=viz)
    best_model = trainer_m.train()
    ### 

    ''' IF YOU WANT TO TRAIN FROM STRATCH COMMENT OUT THIS LINES. 
    criterion = CrossEntropyLoss()
    trainer_m = Trainer(model, criterion, train_loader, val_loader, optimizer, num_epoch=200, patience=PATIENCE,batch_size=BATCH_SIZE,lr_scheduler= lr_sche,display=viz)
    best_model = trainer_m.train()
'''

...

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 0.625s
your url is: https://soft-swan-75.loca.lt
your url is: https://tasty-mole-49.loca.lt
your url is: https://fast-insect-61.loca.lt


Setting up a new session...
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Epoch 0,phase: Training: 1it [00:00, 31775.03it/s, train_loss : ?, train_acc: ?]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
Epoch 0,phase: Training: 1it [00:12, 12.66s/it, train acc:  0.190,train loss:  2.348]/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being depreca

New best loss, loss is:  2.258782970628073 acc is:  0.2813411078717201


Epoch 2,phase: Training: 3it [00:43, 15.22s/it, val acc:  0.366,val loss:  2.185]  

New best loss, loss is:  2.18479355213254 acc is:  0.36676384839650145


...

Epoch 199,phase: Training: 200it [1:08:48, 20.91s/it, val acc:  0.626,val loss:  1.912]  

Not a better score


Epoch 199,phase: Validation: 200it [1:09:09, 20.75s/it, val acc:  0.632,val loss:  1.911]

Not a better score
Training ended returning the best model
Best val acc: 0.6603498542274052, Best val loss: 1.8855652232502782, Best train acc: 0.798941798941799, Best train loss: 1.7450914206887096 


Ellipsis